<a href="https://colab.research.google.com/github/gigerbytes/simulate_a_self_driving_car/blob/master/Train_Car_Model_with_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simulate a self driving car

This code trains a model that can be used to simulate a self driving car through Udacity's open-sourced [Self Driving Car Simulator.](https://github.com/udacity/self-driving-car-sim)

The notebook was ported from [Siraj Raval's How to Simulate a Self Driving Car repository](https://github.com/llSourcell/How_to_simulate_a_self_driving_car
) to make developing and editing the model easier.

## Instructions
Just upload your `data` folder as a .zip file into colab and run the notebook!


In [0]:
import pandas as pd # data analysis toolkit - create, read, update, delete datasets
import numpy as np #matrix math
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split #to split out training and testing data 
#keras is a high level wrapper on top of tensorflow (machine learning library)
#The Sequential container is a linear stack of layers
from keras.models import Sequential
#popular optimization strategy that uses gradient descent 
from keras.optimizers import Adam
#to save our model periodically as checkpoints for loading later
from keras.callbacks import ModelCheckpoint
#what types of layers do we want our model to have?
from keras.layers import Lambda, Conv2D, MaxPooling2D, Dropout, Dense, Flatten
#helper class to define input shape and generate training images given image paths & steering angles
from utils import INPUT_SHAPE, batch_generator
#for command line arguments
import argparse
#for reading files
import os

In [0]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [0]:
config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 1} ) 
sess = tf.Session(config=config) 
keras.backend.set_session(sess)

In [0]:
!unzip data.zip


Archive:  data.zip
   creating: data/
  inflating: data/.DS_Store          
   creating: __MACOSX/
   creating: __MACOSX/data/
  inflating: __MACOSX/data/._.DS_Store  
   creating: data/IMG/
  inflating: data/IMG/right_2019_06_09_03_34_51_752.jpg  
  inflating: data/IMG/center_2019_06_09_03_35_13_512.jpg  
  inflating: data/IMG/center_2019_06_09_03_34_28_167.jpg  
  inflating: data/IMG/right_2019_06_09_03_33_54_314.jpg  
  inflating: data/IMG/center_2019_06_09_03_34_21_548.jpg  
  inflating: data/IMG/right_2019_06_09_03_34_24_316.jpg  
  inflating: data/IMG/left_2019_06_09_03_35_03_128.jpg  
  inflating: data/IMG/left_2019_06_09_03_35_24_463.jpg  
  inflating: data/IMG/left_2019_06_09_03_35_22_383.jpg  
  inflating: data/IMG/left_2019_06_09_03_35_13_896.jpg  
  inflating: data/IMG/center_2019_06_09_03_34_02_433.jpg  
  inflating: data/IMG/center_2019_06_09_03_34_11_282.jpg  
  inflating: data/IMG/center_2019_06_09_03_34_16_765.jpg  
  inflating: data/IMG/left_2019_06_09_03_35_07_295.jp

In [0]:


def load_data(args):
    """
    Load training data and split it into training and validation set
    """
    #reads CSV file into a single dataframe variable
    data_df = pd.read_csv(os.path.join(os.getcwd(), args['data_dir'], 'driving_log.csv'), names=['center', 'left', 'right', 'steering', 'throttle', 'reverse', 'speed'])

    #yay dataframes, we can select rows and columns by their names
    #we'll store the camera images as our input data
    X = data_df[['center', 'left', 'right']].values
    #and our steering commands as our output data
    y = data_df['steering'].values

    #now we can split the data into a training (80), testing(20), and validation set
    #thanks scikit learn
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=args['test_size'], random_state=0)

    return X_train, X_valid, y_train, y_valid


In [0]:

def build_model(args):
    """
    NVIDIA model used
    Image normalization to avoid saturation and make gradients work better.
    Convolution: 5x5, filter: 24, strides: 2x2, activation: ELU
    Convolution: 5x5, filter: 36, strides: 2x2, activation: ELU
    Convolution: 5x5, filter: 48, strides: 2x2, activation: ELU
    Convolution: 3x3, filter: 64, strides: 1x1, activation: ELU
    Convolution: 3x3, filter: 64, strides: 1x1, activation: ELU
    Drop out (0.5)
    Fully connected: neurons: 100, activation: ELU
    Fully connected: neurons: 50, activation: ELU
    Fully connected: neurons: 10, activation: ELU
    Fully connected: neurons: 1 (output)

    # the convolution layers are meant to handle feature engineering
    the fully connected layer for predicting the steering angle.
    dropout avoids overfitting
    ELU(Exponential linear unit) function takes care of the Vanishing gradient problem. 
    """
    model = Sequential()
    model.add(Lambda(lambda x: x/127.5-1.0, input_shape=INPUT_SHAPE))
    model.add(Conv2D(24, 5, 5, activation='elu', subsample=(2, 2)))
    model.add(Conv2D(36, 5, 5, activation='elu', subsample=(2, 2)))
    model.add(Conv2D(48, 5, 5, activation='elu', subsample=(2, 2)))
    model.add(Conv2D(64, 3, 3, activation='elu'))
    model.add(Conv2D(64, 3, 3, activation='elu'))
    model.add(Dropout(args['keep_prob']))
    model.add(Flatten())
    model.add(Dense(100, activation='elu'))
    model.add(Dense(50, activation='elu'))
    model.add(Dense(10, activation='elu'))
    model.add(Dense(1))
    model.summary()

    return model



In [0]:

def train_model(model, args, X_train, X_valid, y_train, y_valid):
    """
    Train the model
    """
    #Saves the model after every epoch.
    #quantity to monitor, verbosity i.e logging mode (0 or 1), 
    #if save_best_only is true the latest best model according to the quantity monitored will not be overwritten.
    #mode: one of {auto, min, max}. If save_best_only=True, the decision to overwrite the current save file is
    # made based on either the maximization or the minimization of the monitored quantity. For val_acc, 
    #this should be max, for val_loss this should be min, etc. In auto mode, the direction is automatically
    # inferred from the name of the monitored quantity.
    checkpoint = ModelCheckpoint('model-{epoch:03d}.h5',
                                 monitor='val_loss',
                                 verbose=0,
                                 save_best_only=args['save_best_only'],
                                 mode='auto')

    #calculate the difference between expected steering angle and actual steering angle
    #square the difference
    #add up all those differences for as many data points as we have
    #divide by the number of them
    #that value is our mean squared error! this is what we want to minimize via
    #gradient descent
    model.compile(loss='mean_squared_error', optimizer=Adam(lr=args['learning_rate']))

    #Fits the model on data generated batch-by-batch by a Python generator.

    #The generator is run in parallel to the model, for efficiency. 
    #For instance, this allows you to do real-time data augmentation on images on CPU in 
    #parallel to training your model on GPU.
    #so we reshape our data into their appropriate batches and train our model simulatenously
    model.fit_generator(batch_generator(args['data_dir'], X_train, y_train, args['batch_size'], True),
                        args['samples_per_epoch'],
                        args['nb_epoch'],
                        max_q_size=1,
                        validation_data=batch_generator(args['data_dir'], X_valid, y_valid, args['batch_size'], False),
                        nb_val_samples=len(X_valid),
                        callbacks=[checkpoint],
                        verbose=1)


In [0]:

#for command line args
def s2b(s):
    """
    Converts a string to boolean value
    """
    s = s.lower()
    return s == 'true' or s == 'yes' or s == 'y' or s == '1'



In [0]:

def main():

    sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
    print(sess)
  
    args = {
      "test_size":0.2,
      "keep_prob":0.5,
      "nb_epoch":10,
      "samples_per_epoch" : 20000,
      "batch_size": 50,
      "save_best_only": 'true',
      "learning_rate": 1.0e-3,
      "data_dir":"data"
    }
    
    
    #print parameters
    print('-' * 30)
    print('Parameters')
    print('-' * 30)
    for key, value in args.items():
        print('{:<20} := {}'.format(key, value))
    print('-' * 30)

    #load data
    data = load_data(args)
    #build model
    model = build_model(args)
    #train model on data, it saves as model.h5 
    train_model(model, args, *data)


In [0]:
main()

------------------------------
Parameters
------------------------------
test_size            := 0.2
keep_prob            := 0.5
nb_epoch             := 10
samples_per_epoch    := 20000
batch_size           := 50
save_best_only       := true
learning_rate        := 0.001
data_dir             := data
------------------------------
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_11 (Lambda)           (None, 66, 200, 3)        0         
_________________________________________________________________
conv2d_43 (Conv2D)           (None, 31, 98, 24)        1824      
_________________________________________________________________
conv2d_44 (Conv2D)           (None, 14, 47, 36)        21636     
_________________________________________________________________
conv2d_45 (Conv2D)           (None, 5, 22, 48)         43248     
_________________________________________________________________
conv2d_4

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(24, (5, 5), activation="elu", strides=(2, 2))`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(36, (5, 5), activation="elu", strides=(2, 2))`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (5, 5), activation="elu", strides=(2, 2))`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="elu")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="elu")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is 

Epoch 1/10
  185/20000 [..............................] - ETA: 51:23 - loss: 0.0436